In [22]:
#importing some useful packages
import math
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2

from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

#class for averaging both locally (between line pieces) and globally (between images)
class line_stats:
    def __init__(self, new_weight=1, max_weight=10):
        self.x = 0
        self.y = 0
        self.slope = 0
        self.weight = 0
        self.new_weight = new_weight
        self.max_weight = max_weight

    def add(self, x, y, slope):
        self.x = (self.weight * self.x + self.new_weight * x) / (self.weight + self.new_weight)
        self.y = (self.weight * self.y + self.new_weight * y) / (self.weight + self.new_weight)
        self.slope = (self.weight * self.slope + self.new_weight * slope) / (self.weight + self.new_weight)
        if self.weight<self.max_weight:
          self.weight+=self.new_weight

#class for detecting line lanes        
class line_detector:    
    def __init__(self):
        #storing the global statistics as well as the vertices for both lanes
        self.lanes = [line_stats() for _ in range(2)]
        self.vertices = np.array([])
        
    def grayscale(self, img):
        """Applies the Grayscale transform
        This will return an image with only one color channel
        but NOTE: to see the returned image as grayscale
        (assuming your grayscaled image is called 'gray')
        you should call plt.imshow(gray, cmap='gray')"""
        return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
    def canny(self, img, low_threshold, high_threshold):
        """Applies the Canny transform"""
        return cv2.Canny(img, low_threshold, high_threshold)

    def gaussian_blur(self, img, kernel_size):
        """Applies a Gaussian Noise kernel"""
        return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

    def region_of_interest(self, img, vertices):
        """
        Applies an image mask.
        
        Only keeps the region of the image defined by the polygon
        formed from `vertices`. The rest of the image is set to black.
        """
        #defining a blank mask to start with
        mask = np.zeros_like(img)   
    
        #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
        if len(img.shape) > 2:
            channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
            ignore_mask_color = (255,) * channel_count
        else:
            ignore_mask_color = 255
        
        #filling pixels inside the polygon defined by "vertices" with the fill color    
        cv2.fillPoly(mask, vertices, ignore_mask_color)
    
        #returning the image only where mask pixels are nonzero
        masked_image = cv2.bitwise_and(img, mask)
        return masked_image


    def draw_lines(self, img, lines, y_limit, color=[255, 0, 0], thickness=10):
        act_lanes = [line_stats(1, 100) for _ in range(2)] #for averaging local pieces
        y_size = img.shape[1]
            
        for line in lines:
          for x1,y1,x2,y2 in line:
            slope = (x2-x1)/(y2-y1)
            if abs(slope)>2: #discarding line pieces with >=45 degrees of slope (and the inf values in case div by z)                
                continue
            if slope<0:      #updating local lane stats               
                act_lanes[0].add((x1+x2)/2, (y1+y2)/2, slope)
            else: 
                act_lanes[1].add((x1+x2)/2, (y1+y2)/2, slope)
                
        for i in range(2):
            pic_lane = act_lanes[i]
            global_lane = self.lanes[i]            
            if pic_lane.weight>0:  #updating global lane stats
               global_lane.add(pic_lane.x, pic_lane.y, pic_lane.slope)
            x1 = int(global_lane.x+(y_size-global_lane.y)*global_lane.slope)
            x2 = int(global_lane.x+(y_limit-global_lane.y)*global_lane.slope)
            cv2.line(img, (x1, y_size), (x2, y_limit), color, thickness)       

    def hough_lines(self, img, rho, theta, threshold, min_line_len, max_line_gap, y_limit):
        """
        `img` should be the output of a Canny transform.
            
        Returns an image with hough lines drawn.
        """
        lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        self.draw_lines(line_img, lines, y_limit)
        return line_img

    def weighted_img(self, img, initial_img, α=0.8, β=1., λ=0.):
        """
        `img` is the output of the hough_lines(), An image with lines drawn on it.
        Should be a blank image (all black) with lines drawn on it.
    
        `initial_img` should be the image before any processing.
    
        The result image is computed as follows:
    
        initial_img * α + img * β + λ
        NOTE: initial_img and img must be the same shape!
        """
        return cv2.addWeighted(initial_img, α, img, β, λ)
    
    def set_roi(self, p1, p2, p3, p4):
        self.vertices = np.array([[p1, p2, p3, p4]], dtype=np.int32)
    
    def process_image(self, image):
        # NOTE: The output you return should be a color image (3 channel) for processing video below
        # TODO: put your pipeline here,
        # you should return the final output (image where lines are drawn on lanes)
        img_gray  = self.grayscale(image)
        img_blur  = self.gaussian_blur(img_gray, 5)
        img_canny = self.canny(img_blur, 10, 250)
        
        #this does not really belong here but this way it works with both video sizes without updating anything
        if not self.vertices.any():
          if image.shape[0]==540:
            self.set_roi((0, 540), (450, 320), (520, 320), (900, 540))
          else: 
            self.set_roi((200, 660), (600, 450), (780, 450), (1100, 660))
        
        img_roi = self.region_of_interest(img_canny, self.vertices)
        img_hough = self.hough_lines(img_roi, 2, np.pi/180, 25, 30, 30, self.vertices[0][1][1])
        result = self.weighted_img(img_hough, image)        
        
        return result

def detect_lane_lines(input_clip, output_clip):   
    test_clip = VideoFileClip(input_clip)
    detector = line_detector()
    new_clip = test_clip.fl_image( detector.process_image )
    %time new_clip.write_videofile(output_clip, audio=False)

In [23]:
#input = "test_videos/solidWhiteRight.mp4"
input = "test_videos/solidYellowLeft.mp4"
#input = "test_videos/challenge.mp4"
output = "test_output.mp4"

detect_lane_lines(input, output)

[MoviePy] >>>> Building video test_output.mp4
[MoviePy] Writing video test_output.mp4


100%|█████████▉| 681/682 [00:17<00:00, 39.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_output.mp4 

CPU times: user 5.65 s, sys: 490 ms, total: 6.14 s
Wall time: 17.9 s


In [24]:
HTML("""
<video width="640" height="300" controls>
  <source src={0} type="video/mp4">
</video>
""".format(output))

#Test with images
import os
os.listdir("test_images/")

#reading in an image
#image = mpimg.imread('test_images/solidWhiteRight.jpg')
#image = mpimg.imread('test_images/solidYellowLeft.jpg')
#image = mpimg.imread('test_images/solidYellowCurve.jpg')
image = mpimg.imread('test_images/solidYellowCurve2.jpg')
#image = mpimg.imread('test_images/bscap0001.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)


detector = line_detector()
result = detector.process_image(image)

plt.imshow(result)